In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Input, LSTM, Dense

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open('/content/drive/My Drive/household_power_consumption.txt', 'r') as f:
  data = f.read()

In [ ]:
data_lines = data.split('\n')[1:]
df = pd.DataFrame([line.split(';') for line in data_lines if line])

df.columns = ['Date', 'Time', 'Global_active_power', 'Global_reactive_power',
              'Voltage', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2',
              'Sub_metering_3']

df.head()

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,16/12/2006,17:24:00,4.216,0.418,234.840,18.400,0.000,1.000,17.000
1,16/12/2006,17:25:00,5.360,0.436,233.630,23.000,0.000,1.000,16.000
2,16/12/2006,17:26:00,5.374,0.498,233.290,23.000,0.000,2.000,17.000
3,16/12/2006,17:27:00,5.388,0.502,233.740,23.000,0.000,1.000,17.000
4,16/12/2006,17:28:00,3.666,0.528,235.680,15.800,0.000,1.000,17.000


In [ ]:
df = df.drop(['Date', 'Time'], axis=1)

In [ ]:
df.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,4.216,0.418,234.840,18.400,0.000,1.000,17.000
1,5.360,0.436,233.630,23.000,0.000,1.000,16.000
2,5.374,0.498,233.290,23.000,0.000,2.000,17.000
3,5.388,0.502,233.740,23.000,0.000,1.000,17.000
4,3.666,0.528,235.680,15.800,0.000,1.000,17.000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1906699 entries, 0 to 2075258
Data columns (total 7 columns):
 #   Column                 Dtype 
---  ------                 ----- 
 0   Global_active_power    object
 1   Global_reactive_power  object
 2   Voltage                object
 3   Global_intensity       object
 4   Sub_metering_1         object
 5   Sub_metering_2         object
 6   Sub_metering_3         object
dtypes: object(7)
memory usage: 116.4+ MB


In [ ]:
df.replace('?', np.nan, inplace=True)


In [ ]:
print("Missing Values:\n", df.isnull().sum())
print("\nDuplicate Rows:", df.duplicated().sum())

Missing Values:
 Global_active_power      0
Global_reactive_power    0
Voltage                  0
Global_intensity         0
Sub_metering_1           0
Sub_metering_2           0
Sub_metering_3           0
dtype: int64

Duplicate Rows: 168560


In [ ]:
df = df.drop_duplicates()
df.dropna(inplace=True)
print("Missing Values:\n", df.isnull().sum())
print("\nDuplicate Rows:", df.duplicated().sum())

Missing Values:
 Global_active_power      0
Global_reactive_power    0
Voltage                  0
Global_intensity         0
Sub_metering_1           0
Sub_metering_2           0
Sub_metering_3           0
dtype: int64

Duplicate Rows: 0


In [ ]:
# Normalisasi data
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df[['Global_active_power']])

# Membuat dataset untuk LSTM
def create_sequences(data, time_steps=10):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:i+time_steps])
        y.append(data[i+time_steps])
    return np.array(X), np.array(y)

time_steps = 10
X, y = create_sequences(df_scaled, time_steps)

# Memisahkan data menjadi train dan test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

# Membuat model LSTM
model = Sequential([
    Input(shape=(time_steps, 1)),
    LSTM(50, activation='relu', return_sequences=True),
    LSTM(50, activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')

# Melatih model
model.fit(X_train, y_train, epochs=5, batch_size=256, validation_data=(X_test, y_test))

# Prediksi
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)
print("Prediksi konsumsi daya:", predictions)


Epoch 1/5
5959/5959 ━━━━━━━━━━━━━━━━━━━━ 323s 53ms/step - loss: 0.0011 - val_loss: 5.9961e-04
Epoch 2/5
5959/5959 ━━━━━━━━━━━━━━━━━━━━ 316s 52ms/step - loss: 5.9623e-04 - val_loss: 5.8106e-04
Epoch 3/5
5959/5959 ━━━━━━━━━━━━━━━━━━━━ 305s 51ms/step - loss: 5.8897e-04 - val_loss: 5.6862e-04
Epoch 4/5
5959/5959 ━━━━━━━━━━━━━━━━━━━━ 327s 52ms/step - loss: 5.6538e-04 - val_loss: 5.5343e-04
Epoch 5/5
5959/5959 ━━━━━━━━━━━━━━━━━━━━ 312s 52ms/step - loss: 5.5027e-04 - val_loss: 5.4362e-04
11917/11917 ━━━━━━━━━━━━━━━━━━━━ 52s 4ms/step
Prediksi konsumsi daya: [[0.32601172]
 [0.44716045]
 [0.40880793]
 ...
 [0.31787622]
 [0.26524115]
 [0.31759754]]


In [ ]:
model.save('lstm_model.keras')
model = load_model('lstm_model.keras')

def get_user_input():
    device_power = {
        "mesin pencuci piring": 1.5,  # kWh per unit (perkiraan)
        "oven": 2.0,
        "microwave": 1.2,
        "mesin cuci": 0.5,
        "pengering": 1.5,
        "kulkas": 0.8,
        "lampu": 0.1,
        "pemanas air listrik": 1.8,
        "AC": 1.5
    }

    total_power = {
        "sub_metering_1": 0,  # Kitchen appliances
        "sub_metering_2": 0,  # Laundry room appliances
        "sub_metering_3": 0   # Water heater and AC
    }

    print("Masukkan perangkat yang digunakan (ketik 'selesai' untuk berhenti):")

    for i in range(5):  # Membatasi input maksimal 5 perangkat
        device = input(f"Masukkan perangkat {i+1}: ")
        if device.lower() == 'selesai':
            break
        if device not in device_power:
            print("Perangkat tidak dikenali. Masukkan perangkat yang valid.")
            continue
        count = input(f"Masukkan jumlah {device} yang digunakan: ")
        try:
            count = int(count)
            if device in ["mesin pencuci piring", "oven", "microwave"]:
                total_power["sub_metering_1"] += device_power[device] * count
            elif device in ["mesin cuci", "pengering", "kulkas", "lampu"]:
                total_power["sub_metering_2"] += device_power[device] * count
            elif device in ["pemanas air listrik", "AC"]:
                total_power["sub_metering_3"] += device_power[device] * count
        except ValueError:
            print("Input jumlah perangkat tidak valid, harus berupa angka.")

    print(f"Total konsumsi daya dapur: {total_power['sub_metering_1']:.2f} kWh")
    print(f"Total konsumsi daya ruang cuci: {total_power['sub_metering_2']:.2f} kWh")
    print(f"Total konsumsi daya pemanas air dan AC: {total_power['sub_metering_3']:.2f} kWh")
    return total_power

# Fungsi untuk prediksi
def make_prediction(user_power):
    total_power = sum(user_power.values())  # Total daya dari semua kategori
    scaler = MinMaxScaler()
    data_scaled = scaler.fit_transform(np.array(total_power).reshape(-1, 1))  # Normalisasi total daya
    # Mengubah total daya menjadi array dengan dimensi yang sesuai
    X_input = np.array(data_scaled).reshape(1, 1, 1)  # Dimensi yang benar untuk model (1 sample, 1 timestep, 1 feature)
    prediction = model.predict(X_input)
    predicted_power = scaler.inverse_transform(prediction)
    predicted_value = predicted_power[0][0]

    print(f"Prediksi konsumsi daya untuk 10 menit ke depan: {predicted_value:.2f} kWh")

    # Menambahkan rekomendasi
    if predicted_value < 2:
        recommendation = "Konsumsi daya rendah, Anda bisa menghemat lebih banyak energi."
    elif 2 <= predicted_value < 5:
        recommendation = "Konsumsi daya sedang, pertimbangkan untuk mengurangi penggunaan energi."
    else:
        recommendation = "Konsumsi daya tinggi, disarankan untuk mengurangi penggunaan energi."

    print(recommendation)

# Mengambil input dan membuat prediksi
user_power = get_user_input()
make_prediction(user_power)

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 10 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Masukkan perangkat yang digunakan (ketik 'selesai' untuk berhenti):
Masukkan perangkat 1: AC
Masukkan jumlah AC yang digunakan: 2
Masukkan perangkat 2: lampu
Masukkan jumlah lampu yang digunakan: 4
Masukkan perangkat 3: kulkas
Masukkan jumlah kulkas yang digunakan: 1
Masukkan perangkat 4: oven
Masukkan jumlah oven yang digunakan: 1
Masukkan perangkat 5: microwave
Masukkan jumlah microwave yang digunakan: 1
Total konsumsi daya dapur: 3.20 kWh
Total konsumsi daya ruang cuci: 1.20 kWh
Total konsumsi daya pemanas air dan AC: 3.00 kWh


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 394ms/step
Prediksi konsumsi daya untuk 10 menit ke depan: 7.43 kWh
Konsumsi daya tinggi, disarankan untuk mengurangi penggunaan energi.
